In [1]:
#Dependencies and Setup
import pandas as pd
import numpy as np

#File to Load
file_to_load= "purchase_data.csv"

#Read Purchasing File and store into Pandas data frame
purchase_data_pd= pd.read_csv(file_to_load)

#Check file was read correctly
purchase_data_pd.head()


,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [2]:
#Player Count
TotalPlayers=len(purchase_data_pd["SN"].unique())
totalplayers_df= pd.DataFrame({"TotalPlayers":[TotalPlayers]})
totalplayers_df


,TotalPlayers
0,576


In [3]:
#Purchasing Analysis(Total)

#Data manipulation
NumUniqueItems= len(purchase_data_pd['Item Name'].value_counts())
AvgPrice= purchase_data_pd['Price'].mean()
NumPurchases= (purchase_data_pd['Item Name']).count()
Total_Revenue= purchase_data_pd['Price'].sum()

#Create Data Frame
Purchasing_analysis=pd.DataFrame({"Number of Unique Items":[NumUniqueItems],
                         "Average Price":[AvgPrice],
                         "Number of Purchases":[NumPurchases],
                         "Total Revenue":[Total_Revenue],

})


#Format All Columns
Purchasing_analysis["Average Price"]= Purchasing_analysis["Average Price"].map("${:,.2f}".format)
Purchasing_analysis["Total Revenue"]= Purchasing_analysis["Total Revenue"].map("${:,.2f}".format)

#Reorder Columns
Purchase_Total=Purchasing_analysis[["Number of Unique Items","Average Price","Number of Purchases","Total Revenue"]]

Purchase_Total


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


In [4]:
#Gender Demographics 

#Remove duplicate players
duplicate=purchase_data_pd.drop_duplicates(subset='SN',keep= "first")
#Count of Male Players, Female Players, and Other/Non-Disclosed
TotalCount=duplicate["Gender"].count()
MaleTotal=duplicate["Gender"].value_counts()['Male']
FemaleTotal=duplicate["Gender"].value_counts()['Female']
OtherTotal=duplicate["Gender"].value_counts()["Other / Non-Disclosed"]

#Count to Percentage
MalePercentage=(MaleTotal/TotalCount)*100
FemalePercentage=(FemaleTotal/TotalCount)*100
OtherPercentage=(OtherTotal/TotalCount)*100

#New DataFrame
gender_demographic_df=pd.DataFrame({"":['Male','Female','Other/Non-Disclosed'],
                                    "Total Count":[MaleTotal, FemaleTotal, OtherTotal],
                                   "Percentage of Players":[MalePercentage, FemalePercentage, OtherPercentage]})

#Format DataFrame
gender_demographic_df["Percentage of Players"]=gender_demographic_df["Percentage of Players"].map("{:,.2f}%".format)
gender_demographic_df= gender_demographic_df.set_index('')
gender_demographic_df


,Total Count,Percentage of Players
,,
Male,484,84.03%
Female,81,14.06%
Other/Non-Disclosed,11,1.91%


In [16]:
#Purchasing Analysis (Gender)

# Group by Gender
grouped_gender_df = purchase_data_pd.groupby(["Gender"])

# Data Manipulation
Purchase_Count = grouped_gender_df["SN"].count()
Purchase_Price = grouped_gender_df["Price"].mean()
Purchase_Norm = grouped_gender_df["Price"].sum()

# Normalize data by deleting all duplicates adn resort
duplicate = purchase_data_pd.drop_duplicates(subset='SN', keep="first")
grouped_dup = duplicate.groupby(["Gender"])

# Find normalized data
Purchase_Value = (grouped_gender_df["Price"].sum() / grouped_dup["SN"].count())

# Create new DataFrame
Gender_Purchase = pd.DataFrame({"Purchase Count": Purchase_Count,
                              "Average Purchase Price": Purchase_Price,
                              "Total Purchase Value": Purchase_Value,
                              "Avg Total Purchase per Person": Purchase_Norm})

# DataFrame formatting
Gender_Purchase["Average Purchase Price"] = Gender_Purchase["Average Purchase Price"].map("${:.2f}".format)
Gender_Purchase["Total Purchase Value"] = Gender_Purchase["Total Purchase Value"].map("${:.2f}".format)
Gender_Purchase["Avg Total Purchase per Person"] = Gender_Purchase["Avg Total Purchase per Person"].map("${:.2f}".format)
Gender_Purchase = Gender_Purchase[["Purchase Count", "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"]]


Gender_Purchase

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$4.47,$361.94
Male,652,$3.02,$4.07,$1967.64
Other / Non-Disclosed,15,$3.35,$4.56,$50.19


In [17]:
#Age Demographics

#Establish bins for ages
Bins = [0, 9, 14, 19, 24,29,34,39,100]
GroupNames = ['<10', '10-14', '15-19', '20-24','25-29','30-34','35-39','40+']

# Segment and sort age values into bins established above
purchase_data_pd["Age Group"] = pd.cut(purchase_data_pd["Age"],Bins, labels=GroupNames)
purchase_data_pd

# Create new data frame with the added "Age Group" and group it
AgeGroup = purchase_data_pd.groupby("Age Group")

# Count total players by age category
TotalAgeCount = AgeGroup["SN"].nunique()

# Calculate percentages by age category 
AgePercentage = (TotalAgeCount/TotalCount) * 100

# Create dataframe with obtained values
AgeDemographics = pd.DataFrame({"Total Count": TotalAgeCount,
                                "Percentage of Players": AgePercentage,
                                 })

# Format percentage with two decimal places 
AgeDemographics.style.format({"Percentage of Players":"{:,.2f}%"})


,Total Count,Percentage of Players
Age Group,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


In [18]:
#Purchasing Analysis (Age)

# Count purchases by age group
PurchaseCountAge = AgeGroup["Purchase ID"].count()

# Obtain average purchase price by age group 
AvgPurchasePrice = AgeGroup["Price"].mean()

# Calculate total purchase value by age group 
TotalPurchaseAge = AgeGroup["Price"].sum()

# Calculate the average purchase per person by age
AvgPurchaseAge = TotalPurchaseAge/TotalAgeCount

# Create dataframe with obtained values
AgeDemographics = pd.DataFrame({"Purchase Count": PurchaseCountAge,
                                 "Average Purchase Price": AvgPurchasePrice,
                                 "Total Purchase Value":TotalPurchaseAge,
                                 "Average Purchase Total per Person": AvgPurchaseAge})

# Format
AgeDemographics.style.format({"Average Purchase Price":"${:,.2f}",
                               "Total Purchase Value":"${:,.2f}",
                               "Average Purchase Total per Person":"${:,.2f}"})




,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Age Group,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [25]:
#Top Spenders

# Group purchase data 
Spenders = purchase_data_pd.groupby("SN")
Spenders.head()
# Calculate the total purchases 
PurchaseCountSpender = Spenders["Purchase ID"].count()

# Calculate the average purchase 
AvgPurchasePrice = Spenders["Price"].mean()

# Calculate purchase total 
PurchaseTotal = Spenders["Price"].sum()

# Create data frame 
TopSpenders = pd.DataFrame({"Purchase Count": PurchaseCountSpender,
                             "Average Purchase Price": AvgPurchasePrice,
                             "Total Purchase Value":PurchaseTotal})
# Preview and descending order
FormattedSpenders = TopSpenders.sort_values(["Total Purchase Value"], ascending=False).head()

# Format
FormattedSpenders.style.format({"Average Purchase Total":"${:,.2f}",
                                 "Average Purchase Price":"${:,.2f}", 
                                 "Total Purchase Value":"${:,.2f}"})



,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [11]:
#Most Popular Items

# Create new data frame 
Items = purchase_data_pd[["Item ID", "Item Name", "Price"]]

# Group by item ID and item name
ItemData = Items.groupby(["Item ID","Item Name"])

# Count the number of times an item has been purchased 
PurchaseCount= ItemData["Price"].count()

# Calculation
PurchaseValue = (ItemData["Price"].sum()) 

# Find individual item price
ItemPrice = PurchaseValue/PurchaseCount

# Create data frame with obtained values
MostPopularItems = pd.DataFrame({"Purchase Count": PurchaseCount, 
                                   "Item Price": ItemPrice,
                                   "Total Purchase Value":PurchaseValue})

# Sort in descending order 
FormatPopularItems= MostPopularItems.sort_values(["Purchase Count"], ascending=False).head()

# Format 
FormatPopularItems.style.format({"Item Price":"${:,.2f}",
                                "Total Purchase Value":"${:,.2f}"})



,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


In [12]:
#Most Profitable Items

#Sort the above table by total purchase value in descending order 

FormatPopularItems= MostPopularItems.sort_values(["Total Purchase Value"], ascending=False).head()

#Format

FormatPopularItems.style.format({"Item Price":"${:,.2f}",
                                 "Total Purchase Value":"${:,.2f}"})


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
